In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

// Initialize the Semantic kernel
var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(                        
        modelId: "phi3:mini",
        apiKey: null,
        endpoint: new Uri("http://localhost:11434")) 
    .Build();

var phi3 = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You are an assistant to support an analyst.
The analyst is working with text coming from the ukrain conflict and contains information in ukraine and russian languages.
The analyst is interested in understanding the sentiment of the text and the entities mentioned in the text.
The analyst is also interested in understanding the context of the text.
The format of the answer should be in markdown.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await phi3.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001

while (true)
{
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    await Chat(userMessage);
}